In [1]:
from matplotlib import pyplot as plt

%matplotlib ipympl

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12
# 用来正常显示中文标签
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['figure.autolayout'] = True
# 用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False 
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

In [2]:
import numpy as np
import pandas as pd
import pylatex
pd.set_option('precision', 2)

from calculate_increases import *
from parse_data import *

In [3]:
benchmark_max_stack_usages = parse_profile_data_in_directory_to_dataframe(
    parse_json_format_profile_data_directory('benchmark_max_stack_usages', 'stack_size'),
    'max_stack_usage',
    True
)

benchmark_instruction_counts = parse_profile_data_in_directory_to_dataframe(
    parse_benchmark_instruction_counts_directory('benchmark_instruction_counts'),
    'instruction_count',
    True
)

benchmark_writes_on_hottest_stack_addresses = parse_profile_data_in_directory_to_dataframe(
    parse_json_format_profile_data_directory('benchmark_writes_on_hottest_stack_addresses', 'writes'),
    'writes_on_hottest_stack_address',
    True
)

max_number_of_stack_frames_on_a_stack_address = parse_profile_data_in_directory_to_dataframe(
    parse_json_format_profile_data_directory('max_number_of_stack_frames_on_a_stack_address', 'number_of_stack_frames'),
    'max_number_of_stack_frames_on_a_stack_address',
    True
)

writes_on_hottest_address_in_stack_frames = parse_profile_data_in_directory_to_dataframe(
    parse_json_format_profile_data_directory('writes_on_hottest_address_in_stack_frames', 'writes'),
    'writes_on_hottest_address_in_stack_frames',
    True
)

In [4]:
baseline_uwlalloc_loop2recursion_executable_type_renaming = {
    'baseline': 'No WL',
    'uwlalloc': 'DynamicStack',
    'new_loop2recursion-recursion_depth_limit-64': 'Loop2Recursion'
}

In [5]:
baseline_uwlalloc_loop2recursion_benchmark_max_stack_usages = benchmark_max_stack_usages[
    benchmark_max_stack_usages['executable_type'].isin(baseline_uwlalloc_loop2recursion_executable_type_renaming)
].replace(
    to_replace=baseline_uwlalloc_loop2recursion_executable_type_renaming
).sort_values(
    by=['benchmark', 'executable_type']
)

baseline_uwlalloc_loop2recursion_benchmark_max_stack_usages

,executable_type,benchmark,max_stack_usage
233,DynamicStack,average,7.76e+05
223,Loop2Recursion,average,6.30e+05
224,No WL,average,6.24e+05
168,DynamicStack,basicmath,4.19e+05
38,Loop2Recursion,basicmath,4.69e+03
51,No WL,basicmath,2.78e+03
156,DynamicStack,bf,2.34e+05
26,Loop2Recursion,bf,1.74e+04
39,No WL,bf,9.20e+03
157,DynamicStack,bitcnts,2.27e+05


In [6]:
maximum_stack_usages_of_no_wear_leveling_dynamic_stack_and_loop2recursion = calculate_increases(
    baseline_uwlalloc_loop2recursion_benchmark_max_stack_usages,
    'No WL', ['DynamicStack', 'Loop2Recursion'], 'max_stack_usage', lambda executable_type: f'{executable_type} Change'
).to_latex(
    formatters={
        f'{executable_type} Change': '{:,.2%}'.format
        for executable_type in baseline_uwlalloc_loop2recursion_executable_type_renaming.values()
    },
    caption='Maximum Stack Usages of No Wear Leveling, Dynamic Stack, and Loop2Recursion',
    label='table:maximum_stack_usages_of_no_wear_leveling_dynamic_stack_and_loop2recursion',
    index=False
)

print(maximum_stack_usages_of_no_wear_leveling_dynamic_stack_and_loop2recursion)

\begin{table}
\centering
\caption{Maximum Stack Usages of No Wear Leveling, Dynamic Stack, and Loop2Recursion}
\label{table:maximum_stack_usages_of_no_wear_leveling_dynamic_stack_and_loop2recursion}
\begin{tabular}{lrrr}
\toprule
benchmark &    No WL & DynamicStack Change & Loop2Recursion Change \\
\midrule
  average & 6.24e+05 &              24.27\% &                 0.89\% \\
basicmath & 2.78e+03 &          14,961.49\% &                68.39\% \\
       bf & 9.20e+03 &           2,443.39\% &                89.57\% \\
  bitcnts & 2.75e+03 &           8,139.83\% &               148.26\% \\
   crc\_32 & 2.16e+03 &          13,541.85\% &                 8.89\% \\
 dijkstra & 2.58e+03 &           3,209.63\% &               475.16\% \\
      fft & 2.82e+03 &           5,842.61\% &               277.84\% \\
 patricia & 3.17e+03 &          12,592.42\% &               127.27\% \\
  pbmsrch & 2.37e+03 &              56.08\% &               511.49\% \\
    qsort & 7.68e+06 &               1.85\

In [7]:
baseline_uwlalloc_loop2recursion_benchmark_instruction_counts = benchmark_instruction_counts[
    benchmark_instruction_counts['executable_type'].isin(baseline_uwlalloc_loop2recursion_executable_type_renaming)
].replace(
    to_replace=baseline_uwlalloc_loop2recursion_executable_type_renaming
).sort_values(
    by=['benchmark', 'executable_type']
)

baseline_uwlalloc_loop2recursion_benchmark_instruction_counts

,executable_type,benchmark,instruction_count
233,DynamicStack,average,1.28e+09
223,Loop2Recursion,average,6.77e+07
224,No WL,average,5.63e+07
168,DynamicStack,basicmath,1.34e+09
38,Loop2Recursion,basicmath,5.04e+07
51,No WL,basicmath,4.94e+07
156,DynamicStack,bf,3.21e+09
26,Loop2Recursion,bf,6.38e+07
39,No WL,bf,5.19e+07
157,DynamicStack,bitcnts,3.42e+09


In [8]:
instruction_counts_of_no_wear_leveling_dynamic_stack_and_loop2recursion = calculate_increases(
    baseline_uwlalloc_loop2recursion_benchmark_instruction_counts,
    'No WL', ['DynamicStack', 'Loop2Recursion'], 'instruction_count', lambda executable_type: f'{executable_type} Change'
).to_latex(
    formatters={
        f'{executable_type} Change': '{:,.2%}'.format
        for executable_type in baseline_uwlalloc_loop2recursion_executable_type_renaming.values()
    },
    caption='Instruction Counts of No Wear Leveling, Dynamic Stack, and Loop2Recursion',
    label='table:instruction_counts_of_no_wear_leveling_dynamic_stack_and_loop2recursion',
    index=False
)

print(instruction_counts_of_no_wear_leveling_dynamic_stack_and_loop2recursion)

\begin{table}
\centering
\caption{Instruction Counts of No Wear Leveling, Dynamic Stack, and Loop2Recursion}
\label{table:instruction_counts_of_no_wear_leveling_dynamic_stack_and_loop2recursion}
\begin{tabular}{lrrr}
\toprule
benchmark &    No WL & DynamicStack Change & Loop2Recursion Change \\
\midrule
  average & 5.63e+07 &           2,179.84\% &                20.16\% \\
basicmath & 4.94e+07 &           2,618.82\% &                 2.05\% \\
       bf & 5.19e+07 &           6,086.06\% &                22.92\% \\
  bitcnts & 9.13e+07 &           3,640.74\% &                48.78\% \\
   crc\_32 & 4.94e+07 &           8,780.95\% &                25.43\% \\
 dijkstra & 1.03e+08 &             212.18\% &                40.35\% \\
      fft & 3.02e+07 &           3,347.48\% &                 2.83\% \\
 patricia & 7.26e+07 &           2,250.64\% &                 1.23\% \\
  pbmsrch & 1.57e+05 &          10,132.18\% &                13.17\% \\
    qsort & 1.46e+07 &           5,951.21\% & 

In [9]:
baseline_uwlalloc_loop2recursion_benchmark_writes_on_hottest_stack_addresses = benchmark_writes_on_hottest_stack_addresses[
    benchmark_writes_on_hottest_stack_addresses['executable_type'].isin(baseline_uwlalloc_loop2recursion_executable_type_renaming)
].replace(
    to_replace=baseline_uwlalloc_loop2recursion_executable_type_renaming
).sort_values(
    by=['benchmark', 'executable_type']
)

baseline_uwlalloc_loop2recursion_benchmark_writes_on_hottest_stack_addresses

,executable_type,benchmark,writes_on_hottest_stack_address
233,DynamicStack,average,1.09e+06
223,Loop2Recursion,average,1.46e+05
224,No WL,average,1.79e+06
168,DynamicStack,basicmath,7.67e+04
38,Loop2Recursion,basicmath,4.90e+04
51,No WL,basicmath,3.73e+05
156,DynamicStack,bf,9.67e+05
26,Loop2Recursion,bf,1.56e+05
39,No WL,bf,1.28e+06
157,DynamicStack,bitcnts,1.05e+06


In [10]:
writes_on_hottest_stack_address_of_no_wear_leveling_dynamic_stack_and_loop2recursion = calculate_increases(
    baseline_uwlalloc_loop2recursion_benchmark_writes_on_hottest_stack_addresses,
    'No WL', ['DynamicStack', 'Loop2Recursion'], 'writes_on_hottest_stack_address', lambda executable_type: f'{executable_type} Change'
).to_latex(
    formatters={
        f'{executable_type} Change': '{:,.2%}'.format
        for executable_type in baseline_uwlalloc_loop2recursion_executable_type_renaming.values()
    },
    caption='Writes on Hottest Stack Address of No Wear Leveling, Dynamic Stack, and Loop2Recursion',
    label='table:writes_on_hottest_stack_address_of_no_wear_leveling_dynamic_stack_and_loop2recursion',
    index=False
)

print(writes_on_hottest_stack_address_of_no_wear_leveling_dynamic_stack_and_loop2recursion)

\begin{table}
\centering
\caption{Writes on Hottest Stack Address of No Wear Leveling, Dynamic Stack, and Loop2Recursion}
\label{table:writes_on_hottest_stack_address_of_no_wear_leveling_dynamic_stack_and_loop2recursion}
\begin{tabular}{lrrr}
\toprule
benchmark &    No WL & DynamicStack Change & Loop2Recursion Change \\
\midrule
  average & 1.79e+06 &             -39.36\% &               -91.88\% \\
basicmath & 3.73e+05 &             -79.44\% &               -86.88\% \\
       bf & 1.28e+06 &             -24.41\% &               -87.81\% \\
  bitcnts & 6.05e+06 &             -82.64\% &               -98.68\% \\
   crc\_32 & 2.74e+06 &              -0.02\% &               -50.01\% \\
 dijkstra & 4.51e+06 &               0.01\% &               -98.90\% \\
      fft & 3.04e+05 &             -81.05\% &               -89.81\% \\
 patricia & 3.86e+05 &             -74.73\% &               -89.13\% \\
  pbmsrch & 1.46e+03 &             -34.29\% &               -97.12\% \\
    qsort & 5.82e+04